In [1]:
import requests
import re
import numpy as np
import pandas as pd
import os
import json

In [2]:
# refer https://droughtmonitor.unl.edu/WebServiceInfo.aspx#comp
Area = 'CountyStatistics'
Stat_type = 'GetDroughtSeverityStatisticsByAreaPercent'
BASE_URL = f'https://usdmdataservices.unl.edu/api/{Area}/{Stat_type}'
BASE_URL

'https://usdmdataservices.unl.edu/api/CountyStatistics/GetDroughtSeverityStatisticsByAreaPercent'

In [3]:
# Definign dates (M/D/YYYY)
startdate = '1/1/2000'
enddate = '12/31/2020'

In [4]:
# Loading the FIPS codes
codes = pd.read_csv('NY_county_FIPS.csv')
codes.head()

,Counties
0,"{""state"":""NY"",""id"":""36001"",""name"":""Albany Cou..."
1,"{""state"":""NY"",""id"":""36003"",""name"":""Allegany C..."
2,"{""state"":""NY"",""id"":""36005"",""name"":""Bronx Coun..."
3,"{""state"":""NY"",""id"":""36007"",""name"":""Broome Cou..."
4,"{""state"":""NY"",""id"":""36009"",""name"":""Cattaraugu..."


In [5]:
codeslist = []
for i in codes['Counties']:
    dic = json.loads(i)
    codeslist.append(dic['id'])

In [6]:
# Other parameters
master_req = []
for c in codeslist:
    params = {
    'aoi' : c, #FIPS code for county Tompkins
    'startdate' : startdate, # M/D/YYYY
    'enddate' : enddate,
    'statisticsType' : '1' # 1 for traditional or 2 for categorical.
    }
    req = requests.get(BASE_URL, params=params)
    c_data = req.json()
    master_req.append(c_data)

In [8]:
# master_req[:3]

In [10]:
# Saving the json file
save_path = '../../../../../../../Google Drive/Shared drives/Capstone Project/Data/USDM/' \
+ 'CountyUSDM' + '_' + startdate.replace('/','') + '-' + enddate.replace('/','')
save_path

'../../../../../../../Google Drive/Shared drives/Capstone Project/Data/USDM/CountyUSDM_112000-12312020'

In [11]:
os.makedirs(os.path.dirname(save_path), exist_ok=True)
with open(save_path, 'w') as f:
    json.dump(master_req, f)

### Converting the json file to csv

In [14]:
type(master_req)

list

In [15]:
len(master_req)

62

In [16]:
len(master_req[0])

1096

In [17]:
master_req[0]

[{'MapDate': '20201229',
  'FIPS': '36001',
  'County': 'Albany County',
  'State': 'NY',
  'None': '100.00',
  'D0': '0.00',
  'D1': '0.00',
  'D2': '0.00',
  'D3': '0.00',
  'D4': '0.00',
  'ValidStart': '2020-12-29',
  'ValidEnd': '2021-01-04',
  'StatisticFormatID': '1'},
 {'MapDate': '20201222',
  'FIPS': '36001',
  'County': 'Albany County',
  'State': 'NY',
  'None': '100.00',
  'D0': '0.00',
  'D1': '0.00',
  'D2': '0.00',
  'D3': '0.00',
  'D4': '0.00',
  'ValidStart': '2020-12-22',
  'ValidEnd': '2020-12-28',
  'StatisticFormatID': '1'},
 {'MapDate': '20201215',
  'FIPS': '36001',
  'County': 'Albany County',
  'State': 'NY',
  'None': '100.00',
  'D0': '0.00',
  'D1': '0.00',
  'D2': '0.00',
  'D3': '0.00',
  'D4': '0.00',
  'ValidStart': '2020-12-15',
  'ValidEnd': '2020-12-21',
  'StatisticFormatID': '1'},
 {'MapDate': '20201208',
  'FIPS': '36001',
  'County': 'Albany County',
  'State': 'NY',
  'None': '100.00',
  'D0': '0.00',
  'D1': '0.00',
  'D2': '0.00',
  'D3': '0.

In [19]:
master_req[0][0]

{'MapDate': '20201229',
 'FIPS': '36001',
 'County': 'Albany County',
 'State': 'NY',
 'None': '100.00',
 'D0': '0.00',
 'D1': '0.00',
 'D2': '0.00',
 'D3': '0.00',
 'D4': '0.00',
 'ValidStart': '2020-12-29',
 'ValidEnd': '2021-01-04',
 'StatisticFormatID': '1'}

In [18]:
master_req[0][-1]

{'MapDate': '20000104',
 'FIPS': '36001',
 'County': 'Albany County',
 'State': 'NY',
 'None': '100.00',
 'D0': '0.00',
 'D1': '0.00',
 'D2': '0.00',
 'D3': '0.00',
 'D4': '0.00',
 'ValidStart': '2000-01-04',
 'ValidEnd': '2000-01-10',
 'StatisticFormatID': '1'}

### From the above we observe that the data is stored in following format:
#### 1. List of counties - thus 62 entries
#### 2. Each list has 1096 entries corresponding to the USDM weekly data for that county stored in a dictionary format

### Converting the dictionary-list object to numpy array that will eventually be stored in the form of a csv for further EDA and analysis
#### Our final data frame would have:
##### 1. Rows: 1096*62 = 67952, and a header
##### 2. Columns: 13 + 1 namely: 'MapDate', 'FIPS', 'County', 'State', 'None', 'D0', 'D1', 'D2', 'D3', 'D4', 'ValidStart', 'ValidEnd', 'StatisticFormatID'. And 1 would be a unique id which here we can keep a combination of County+ValidStart

In [20]:
# Column name array:
col_names = ['MapDate', 'FIPS', 'County', 'State', 'None', 'D0', 'D1', 'D2', 'D3',\
             'D4', 'ValidStart', 'ValidEnd', 'StatisticFormatID']

In [23]:
usdm_final = pd.DataFrame(columns = col_names)

In [ ]:
index = 0
for county in master_req[:1]:
    for week in county:
        usdm_final.loc[index] = list(week.values())
        index = index+1

In [87]:
usdm_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67952 entries, 0 to 67951
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   MapDate            67952 non-null  object
 1   FIPS               67952 non-null  object
 2   County             67952 non-null  object
 3   State              67952 non-null  object
 4   None               67952 non-null  object
 5   D0                 67952 non-null  object
 6   D1                 67952 non-null  object
 7   D2                 67952 non-null  object
 8   D3                 67952 non-null  object
 9   D4                 67952 non-null  object
 10  ValidStart         67952 non-null  object
 11  ValidEnd           67952 non-null  object
 12  StatisticFormatID  67952 non-null  object
dtypes: object(13)
memory usage: 7.3+ MB


In [88]:
usdm_final

,MapDate,FIPS,County,State,None,D0,D1,D2,D3,D4,ValidStart,ValidEnd,StatisticFormatID
0,20201229,36001,Albany County,NY,100.00,0.00,0.00,0.00,0.00,0.00,2020-12-29,2021-01-04,1
1,20201222,36001,Albany County,NY,100.00,0.00,0.00,0.00,0.00,0.00,2020-12-22,2020-12-28,1
2,20201215,36001,Albany County,NY,100.00,0.00,0.00,0.00,0.00,0.00,2020-12-15,2020-12-21,1
3,20201208,36001,Albany County,NY,100.00,0.00,0.00,0.00,0.00,0.00,2020-12-08,2020-12-14,1
4,20201201,36001,Albany County,NY,100.00,0.00,0.00,0.00,0.00,0.00,2020-12-01,2020-12-07,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
67947,20000201,36123,Yates County,NY,100.00,0.00,0.00,0.00,0.00,0.00,2000-02-01,2000-02-07,1
67948,20000125,36123,Yates County,NY,100.00,0.00,0.00,0.00,0.00,0.00,2000-01-25,2000-01-31,1
67949,20000118,36123,Yates County,NY,100.00,0.00,0.00,0.00,0.00,0.00,2000-01-18,2000-01-24,1
67950,20000111,36123,Yates County,NY,100.00,0.00,0.00,0.00,0.00,0.00,2000-01-11,2000-01-17,1


In [31]:
path = '../../../../../../../Google Drive/Shared drives/Capstone Project/Data/USDM/'
filename = 'CountyUSDM' + '_' + startdate.replace('/','') + '-' + enddate.replace('/','')

In [95]:
# Saving the file
usdm_final.to_csv(path+filename+'.csv', index=False)

### Checking if the file is loaded correctly

In [32]:
dum = pd.read_csv(path+filename+'.csv')

In [34]:
dum.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67952 entries, 0 to 67951
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   MapDate            67952 non-null  int64  
 1   FIPS               67952 non-null  int64  
 2   County             67952 non-null  object 
 3   State              67952 non-null  object 
 4   None               67952 non-null  float64
 5   D0                 67952 non-null  float64
 6   D1                 67952 non-null  float64
 7   D2                 67952 non-null  float64
 8   D3                 67952 non-null  float64
 9   D4                 67952 non-null  float64
 10  ValidStart         67952 non-null  object 
 11  ValidEnd           67952 non-null  object 
 12  StatisticFormatID  67952 non-null  int64  
dtypes: float64(6), int64(3), object(4)
memory usage: 6.7+ MB


In [33]:
dum.head()

,MapDate,FIPS,County,State,None,D0,D1,D2,D3,D4,ValidStart,ValidEnd,StatisticFormatID
0,20201229,36001,Albany County,NY,100.0,0.0,0.0,0.0,0.0,0.0,2020-12-29,2021-01-04,1
1,20201222,36001,Albany County,NY,100.0,0.0,0.0,0.0,0.0,0.0,2020-12-22,2020-12-28,1
2,20201215,36001,Albany County,NY,100.0,0.0,0.0,0.0,0.0,0.0,2020-12-15,2020-12-21,1
3,20201208,36001,Albany County,NY,100.0,0.0,0.0,0.0,0.0,0.0,2020-12-08,2020-12-14,1
4,20201201,36001,Albany County,NY,100.0,0.0,0.0,0.0,0.0,0.0,2020-12-01,2020-12-07,1
